# Domain Specific Code Generation using the FormLang DSL

## Abstract

AI and LLM systems are being trained to perform a variety of complex tasks requiring expertise in many software tools, languages and technology stacks. Some of the challenges in this process involve quality data acquisition in large amounts, handling the increase of model parameter count which leads to increasing compute demands and costs, as well as emerging Data Privacy and Intellctual Property concerns related with using 3rd party cloud services for model training.

In this work we attempt to harness and combine the benefits of Abstraction and Determinism provided by Formal Domain Specific Languages (DSLs) with the innate ability of LLMs to learn new languages and their semantics. We propose a novel generation task called "Domain Specific Code Generation" which involve mapping user requests written in natural language to DSL code.

By utilizing a specially crafted DSL called `FormLang` as a case study, we attempt to lay the groundwork for methods of automated DSL dataset generation, training techniques and performance evaluation, with the end goal of creating an AI system capable of generating Web forms according to a user request.

Our `FormLang` DSL allows expressing the semantics of Web-forms using a simplified syntax that does not require much, if any, Web-programming knowledge and expertise.

Given a user prompt in English describing the desired form and its fields the LLM produces syntactically valid `FormLang` output which is run through the accompanying FormLang parser and a hand-crafted React JSX compiler to output a final implementation of the form in JavaScript and React.

The project includes a live demo which demonstrates the capabilities of the system.


## Referring to this work

If you use this work the following quote is preferred:

```bibtex
@misc{guyor2025dscodegenformlang,
      title={Domain Specific Code Generation using the FormLang DSL},
      author={Guy Or},
      year={2025}
}
```

The official repository of this work is hosted in GitHub at https://github.com/guyo13/Form-Lang **TBD** - Make the repo public

## Project Goals

* Define the Task of Domain Specific Code Generation.
* Create an AI training pipeline for FormLang (implemented as a Juypter notebook) which includes:
    * Automatic FormLang Dataset generation using searching algorithms and heuristics.
    * Baseline model selection and loading from Hugging Face.
    * Dataset Preprocessing and loading.
    * Defining performance KPIs for the system.
    * Model fine tuning using Transformers library
    * Model Adapter training using PEFT library.
    * Model upload to Hugging Face Hub and example usage from the Hub.
    *  **(TBD)** Export to ONNX using Optimum and run on-device using Transformers.js.
* Create the “FormLang” language:
    * Describe the problem domain.
    * Defining a viable minimal syntax and semantics which are research focused rather than completeness focused.
    *  **(TBD)** Implementing a “JavaScript React” compile target.
*  **(TBD)** Create a live demo website:
    * **(TBD)** Users input a prompt.
    * **(TBD)** A FormLang editor is populated with the AI’s code generation results.
    * **(TBD)** The form is rendered alongside the generated code.
*  Discuss the project results:
    * Perfornace and user acceptability.
    * **(TBD)** Viability of the implemented methods for the Domain Specific Code Generation task and generalization to other domains.  
    * **(TBD)** Potential enhancements to the system.
    * **(TBD)** Possible research directions on how to learn from user data.


**(TBD)** - Features marked as TBD are depending on the project's progress and timeline constraints as well as proving the viability of the methods and system.

# Notebook Setup

## About the FormLang DSL and its tooling and codebase

The FormLang DSL is implemented using the [Langium](https://langium.org/) project. Langium is a toolkit written in TypeScript that allows language engineers to create DSLs and quickly iterate over their development lifecycle.

Some of the many features offered by Langium are:

* Parser generation - Langium leverages [Chevrotain](https://chevrotain.io/docs/) to create a fast parser that can parse the DSL code to AST.
* TypeScript AST nodes generation - Langium automatically generates TS types that represent the AST of the language.
* Simplified Validation rules - Langium has built in support for writing and executing validation rules with built in error reporting.
* Language Server Protocol support - Langium implements the LSP protocol which allows code completion and syntax highlighting in many IDEs such as VSCode.

Since we are using Langium to parse the FormLang DSL, some of this project tooling and algorithms are written in TypeScript.

To call our TypeScript code from Python we are using two distinct methods:

* [PythonMonkey](https://pythonmonkey.io/) - A Python package that implements FFI between Python and JavaScript.
* Local HTTP API - For some cases in which the PythonMonkey FFI is too slow, we've implemented a local REST-API server that implements some functions we require (e.g for parsing FormLang code to AST).

### Codebase

The entire codebase is hosted in Github at [FormLang](https://github.com/guyo13/Form-Lang).

### Local Setup and Installation instructions - TODO

### Running from Colab

#### Installing PNPM and Bun

In [3]:
!npm install -g pnpm
!pnpm install bun@latest

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸
added 1 package in 3s
⠸
⠸1 package is looking for funding
⠸  run `npm fund` for details
⠸Progress: resolved 1, reused 0, downloaded 0, added 0
Progress: resolved 1, reused 0, downloaded 1, added 0
Progress: resolved 5, reused 0, downloaded 1, added 0
Progress: resolved 10, reused 0, downloaded 1, added 0
Progress: resolved 11, reused 0, downloaded 1, added 0
Progress: resolved 12, reused 0, downloaded 1, added 0
Packages: +5

Progress: resolved 12, reused 0, downloaded 1, added 0
Progress: resolved 12, reused 0, downloaded 1, added 1
Progress: resolved 12, reused 0, downloaded 1, added 1
Progress: resolved 12, reused 0, downloaded 1, added 1
Progress: resolved 12, reused 0, downloaded 1, added 1
Progress: resolved 12, reused 0, downloaded 1, added 1
Progress: resolved 12, reused 0, downloaded 2, added 1
Progress: resolved 12, reused 0, downloaded 3, added 1
Progress: resolved 12, reused 0, downloaded 4, added 1
Progress: resolved 12, reused 0, downloaded 5, adde

#### Installing PythonMonkey

In [4]:
!pip install pythonmonkey

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.6/288.6 kB 17.4 MB/s eta 0:00:00
  Created wheel for pminit: filename=pminit-1.1.0-cp311-cp311-manylinux_2_35_x86_64.whl size=2127649 sha256=ed91d46baa2c3629c4dc2a35a3fcd3a1bf6feea709ff03298382244ea3bb0fb7
  Stored in directory: /root/.cache/pip/wheels/64/5e/39/2de4476a583c81bbfb9ff2d5dc85aa207866eb1960b06775d7
Successfully built pminit


In [2]:
!git clone -b react_compiler https://github.com/guyo13/Form-Lang.git

Cloning into 'Form-Lang'...
fatal: could not read Username for 'https://github.com': No such device or address


## Project imports

In [ ]:
import asyncio
import time
import re
from pprint import pprint
from transformers import pipeline
from huggingface_hub import login
import pythonmonkey as pm
import pandas as pd
import requests
import torch
formlang_lib = pm.require("../out/cjs/lib/index")

## Login to Huggingface Hub

In [ ]:
login()

## Python Monkey helpers

In [ ]:
def js_dir(something):
    pm.globalThis.console.dir(something)

def js_log(something):
    pm.globalThis.console.log(something)

### Example usage of FormLang library

# Dataset generation

In [ ]:
form_components = """
  component userDetailsContainer {}
  component formContainer {}
  component someOtherContainer {}
  component OtherContainer2 {}
"""
field_components = """
  component myTextBox {
    props {
      textColor
      textSize
      textWeight
      borderColor
    }
  }
  component myCheckbox {
    props {
      size
    }
  }
  component otherTextBox {}
  component counter {
    props {
      style
    }
  }
"""

form_gen = await formlang_lib.newFormGen(formlang_lib.DEFAULT_GENERATOR_HYPER_PARAMETERS, form_components, field_components)

Validating model
Validating model


In [ ]:
a = form_gen.generateForm()

In [ ]:
js_dir(a)

{ '$type': 'Form',
  name: 'xw',
  component: 
   { component: 
      { '$type': 'ComponentDef',
        name: 'formContainer',
        '$cstNode': [Object],
        props: [],
        '$container': [Object],
        '$containerProperty': 'components',
        '$containerIndex': 1 },
     propAssignments: {} },
  children: 
   [ { '$type': 'Field',
       name: 'c',
       component: [Object],
       state: null,
       depth: 1 },
     { '$type': 'Field',
       name: 'e',
       component: [Object],
       state: [Object],
       depth: 1 },
     { '$type': 'Form',
       name: 'A',
       component: [Object],
       children: [Array],
       depth: 1 },
     { '$type': 'Field',
       name: 'o',
       component: [Object],
       state: [Object],
       depth: 1 },
     { '$type': 'Field',
       name: 'pw',
       component: [Object],
       state: [Object],
       depth: 1 } ],
  depth: 0 }


In [ ]:
fl_obj = await formlang_lib.getFormLangStringParser()("""
component comp1 {}
component comp2 {}
form HelloWorld {
    comp comp1
    field MyField {
        comp comp2
    }
}
""")

Validating model


In [ ]:
formlang_lib.hasErrors(fl_obj)

False

In [ ]:
js_dir(fl_obj)

{ parseResult: 
   { value: 
      { '$type': 'Model',
        components: [Array],
        forms: [Array],
        '$cstNode': [Object],
        typeDefs: [],
        '$document': [Circular] },
     lexerErrors: [],
     lexerReport: { diagnostics: [] },
     parserErrors: [] },
  uri: 
   l2 {
     scheme: 'file',
     authority: '',
     path: '/10.form',
     query: '',
     fragment: '',
     _formatted: 'file:///10.form',
     _fsPath: null },
  state: 6,
  references: 
   [ { '$refNode': [Object],
       '$refText': 'comp1',
       ref: [Getter],
       '$nodeDescription': [Getter],
       error: [Getter],
       _ref: [Object],
       _nodeDescription: [Object] },
     { '$refNode': [Object],
       '$refText': 'comp2',
       ref: [Getter],
       '$nodeDescription': [Getter],
       error: [Getter],
       _ref: [Object],
       _nodeDescription: [Object] } ],
  textDocument: [Getter],
  precomputedScopes: MultiMap { map: Map { [Object] => [Array], [Object] => [Array] } },
  

In [ ]:
formlang_lib.serializeAst(fl_obj.parseResult.value, formlang_lib.getServices().FormLang)

'{"$type":"Model","components":[{"$type":"ComponentDef","name":"comp1","props":[]},{"$type":"ComponentDef","name":"comp2","props":[]}],"forms":[{"$type":"Form","name":"HelloWorld","component":{"$type":"FieldComponentDef","componentId":{"$ref":"#/components@0"},"componentPropsKeys":[],"componentPropsValues":[]},"children":[{"$type":"Field","name":"MyField","component":{"$type":"FieldComponentDef","componentId":{"$ref":"#/components@1"},"componentPropsKeys":[],"componentPropsValues":[]}}]}],"typeDefs":[]}'

### Generating the dataset

We start by creating a ProbabilisticSearchFormGenerator object which is implemented in JavaScript, providing the default search hyper parameters and some Component definitions,
later on we will implement a random generator for the component definitions as well.

In [ ]:
form_components = """
  component userDetailsContainer {}
  component formContainer {}
  component someOtherContainer {}
  component OtherContainer2 {}
"""
field_components = """
  component myTextBox {
    props {
      textColor
      textSize
      textWeight
      borderColor
    }
  }
  component myCheckbox {
    props {
      size
    }
  }
  component otherTextBox {}
  component counter {
    props {
      style
    }
  }
"""

form_gen = await formlang_lib.newFormGen(formlang_lib.DEFAULT_GENERATOR_HYPER_PARAMETERS, form_components, field_components)

#### Computing the expected AST

In order to reliablity measure to model's performance, our KPI will be **AST-Accuracy** which is the accuracy measure with respect to the generated code's AST and the expected code's AST.

In [ ]:
def get_ast(code, shouldCheckErrors=False):
    result = requests.post("http://localhost:3000/compute/ast", json={"sourceCode": code, "shouldCheckErrors":shouldCheckErrors}).json()
    if not result["status"] == "ok":
        raise RuntimeError("failed")
    return result["result"]

#### Create a data generation loop for the prompt data

We create a small dataset of 3000 random examples by using the JavaScript FormLang utilities library and the form generator object.
We copy from the form generator's output only the relevant fields which will make up the LLM prompt.

In [ ]:
# TODO - Refactor data generator pipeline as a Python class with easy interfaces

def concat_components_code_with_form_code(form_code, form_components, field_components):
    """Concatenates the form code with the given components code that are referenced inside of it."""
    return f"{form_components}\n{field_components}\n{form_code}"

def create_prompt_data(generation_result, form_components, field_components):
    data = {
        "originalFormCode": concat_components_code_with_form_code(generation_result["serializedForm"], form_components, field_components),
        "modifiedFormCode": concat_components_code_with_form_code(generation_result["serializedModifiedForm"], form_components, field_components),
        "removedNodeEnglish": str(generation_result["removedNodeEnglish"]),
        "removedNodeContextEnglish": str(generation_result["removedNodeContextEnglish"]),
    }
    return data

def create_random_data(form_gen, form_components, field_components, num_examples=3000):
    start_time = time.time()
    DATA = []
    for i in range(num_examples):
        generation_result = formlang_lib.generateRandomFormWithModification(form_gen)
        prompt_data = create_prompt_data(generation_result, form_components, field_components)
        DATA.append(prompt_data)
    # Generate the ast of each example in one batch
    ast_time = time.time()
    for example in DATA:
        resp = get_ast(example["originalFormCode"], True)
        errors = resp.get('errors', [])
        if len(errors) > 0:
            print(f"Error parsing generated code", errors)
            raise RuntimeError("Error parsing generated code")
        example["originalFormAst"] = resp["ast"]
    end_time = time.time()
    print(f"Generation took {end_time - start_time} seconds. Total Ast generation time {end_time - ast_time} seconds")
    return pd.DataFrame(DATA)

In [ ]:
examples = create_random_data(form_gen, form_components, field_components, 3000)
examples.head()

Generation took 9.601897478103638 seconds. Total Ast generation time 9.301260709762573 seconds


,originalFormCode,modifiedFormCode,removedNodeEnglish,removedNodeContextEnglish,originalFormAst
0,\n component userDetailsContainer {}\n compo...,\n component userDetailsContainer {}\n compo...,\ta field whose id is 'Ow' with state of type:...,* is a child of the form whose id is 'i_'.\n* ...,"{""$type"":""Form"",""name"":""i_"",""component"":{""$typ..."
1,\n component userDetailsContainer {}\n compo...,\n component userDetailsContainer {}\n compo...,\ta field whose id is 'vw' with state of type:...,* is a child of the form whose id is 'X'.,"{""$type"":""Form"",""name"":""X"",""component"":{""$type..."
2,\n component userDetailsContainer {}\n compo...,\n component userDetailsContainer {}\n compo...,\ta field whose id is 'Tw' with state of type:...,* is a child of the form whose id is 'xw'.\n* ...,"{""$type"":""Form"",""name"":""xw"",""component"":{""$typ..."
3,\n component userDetailsContainer {}\n compo...,\n component userDetailsContainer {}\n compo...,\t\ta field whose id is 'qw_' with state of ty...,* is a child of the form whose id is 'i'.\n* i...,"{""$type"":""Form"",""name"":""I"",""component"":{""$type..."
4,\n component userDetailsContainer {}\n compo...,\n component userDetailsContainer {}\n compo...,\ta field whose id is 'zw_' with state of type...,* is a child of the form whose id is 'h'.\n* i...,"{""$type"":""Form"",""name"":""h"",""component"":{""$type..."


##### Example generated FormLang code and its AST

In [ ]:
examples["originalFormCode"][1132]

'\n  component userDetailsContainer {}\n  component formContainer {}\n  component someOtherContainer {}\n  component OtherContainer2 {}\n\n\n  component myTextBox {\n    props {\n      textColor\n      textSize\n      textWeight\n      borderColor\n    }\n  }\n  component myCheckbox {\n    props {\n      size\n    }\n  }\n  component otherTextBox {}\n  component counter {\n    props {\n      style\n    }\n  }\n\nform Ew {\n\tcomp OtherContainer2 \n\t\n\tform b {\n\t\tcomp someOtherContainer \n\t\t\n\t\tfield _www {\n\t\t\tcomp counter style="(() => 50.663073554388895)()" as expression\n\t\t\t\n\t\t}\n\t}\n\n\tform rw_w {\n\t\tcomp formContainer \n\t\t\n\t\tfield w {\n\t\t\tcomp myTextBox textWeight="typeof \'hello\'" as expression\n\t\t\t\n\t\t}\n\t\tfield Cw {\n\t\t\tcomp myCheckbox \n\t\t\t\n\t\t}\n\t}\n\n\tform s_ {\n\t\tcomp someOtherContainer \n\t\t\n\t\tfield h_ {\n\t\t\tstate boolean\n\t\t\tcomp counter style="#440b2a"\n\t\t\t\n\t\t}\n\t\tfield Vwww {\n\t\t\tstate string\n\t\t\t

In [ ]:
examples["originalFormAst"][1132]

'{"$type":"Form","name":"Ew","component":{"$type":"FieldComponentDef","componentId":{"$ref":"#/components@3"},"componentPropsKeys":[],"componentPropsValues":[]},"children":[{"$type":"Form","name":"b","component":{"$type":"FieldComponentDef","componentId":{"$ref":"#/components@2"},"componentPropsKeys":[],"componentPropsValues":[]},"children":[{"$type":"Field","name":"_www","component":{"$type":"FieldComponentDef","componentId":{"$ref":"#/components@7"},"componentPropsKeys":[{"$type":"ComponentPropKey","key":"style"}],"componentPropsValues":[{"$type":"ValueExpression","value":"(() => 50.663073554388895)()","isExpression":true}]}}]},{"$type":"Form","name":"rw_w","component":{"$type":"FieldComponentDef","componentId":{"$ref":"#/components@1"},"componentPropsKeys":[],"componentPropsValues":[]},"children":[{"$type":"Field","name":"w","component":{"$type":"FieldComponentDef","componentId":{"$ref":"#/components@4"},"componentPropsKeys":[{"$type":"ComponentPropKey","key":"textWeight"}],"compone

### Creating the prompts

We now create the user prompt which will be used to train the LLM.

We instruct the LLM to generate the original form (the `serializedForm` column) given the modified form, an English description of the removed node and its surrounding context (describing which element is the Form's parent and which elements are its siblings).


In [ ]:
def get_system_prompt():
    return (
        "Your job is to generate valid FormLang code according to the instructions given below:\n"
            "Inspect the following FormLang Form definition, the start of the code will be denoted with ```FormLang and its end with ``` .\n"
            "After inspection, complete the Form's code according to the given a description of a new form element and a description of its location in the form.\n"
            "You may assume that the new form element to be added is always either a 'form' or a 'field'.\n"
            "Your output must be valid and compiler-friendly FormLang code only.\n"
            "If you are unsure of the FormLang syntax, try to infer it from the form code which is given below as an input."
            "Your answer will be evaluated using an AST comparison of your code to the expected code.\n"
            "Assume that the input code is valid and requires no modification other than the NEW code you must generate.\n"
            "You must output plain FormLang code without any additional text or delimiters.\n"
            "You must not change any part of the original input code other than adding the required element.\n"
    )

def create_prompt(row, with_system_prompt=False):
    return (get_system_prompt() if with_system_prompt else "") + (
            "```FormLang\n"
            f"{row['modifiedFormCode']}\n"
            "```\n"
            "The description of the form element you need to add:\n"
            f"{row['removedNodeEnglish']}\n"
            "The description of the context in the form where you should add the element:\n"
            f"The element to be added:\n{row['removedNodeContextEnglish']}\n"
           )


In [ ]:
examples["userPrompt"] = examples.apply(create_prompt, axis=1)
examples.head()

,originalFormCode,modifiedFormCode,removedNodeEnglish,removedNodeContextEnglish,originalFormAst,userPrompt
0,\n component userDetailsContainer {}\n compo...,\n component userDetailsContainer {}\n compo...,\ta field whose id is 'Ow' with state of type:...,* is a child of the form whose id is 'i_'.\n* ...,"{""$type"":""Form"",""name"":""i_"",""component"":{""$typ...",```FormLang\n\n component userDetailsContaine...
1,\n component userDetailsContainer {}\n compo...,\n component userDetailsContainer {}\n compo...,\ta field whose id is 'vw' with state of type:...,* is a child of the form whose id is 'X'.,"{""$type"":""Form"",""name"":""X"",""component"":{""$type...",```FormLang\n\n component userDetailsContaine...
2,\n component userDetailsContainer {}\n compo...,\n component userDetailsContainer {}\n compo...,\ta field whose id is 'Tw' with state of type:...,* is a child of the form whose id is 'xw'.\n* ...,"{""$type"":""Form"",""name"":""xw"",""component"":{""$typ...",```FormLang\n\n component userDetailsContaine...
3,\n component userDetailsContainer {}\n compo...,\n component userDetailsContainer {}\n compo...,\t\ta field whose id is 'qw_' with state of ty...,* is a child of the form whose id is 'i'.\n* i...,"{""$type"":""Form"",""name"":""I"",""component"":{""$type...",```FormLang\n\n component userDetailsContaine...
4,\n component userDetailsContainer {}\n compo...,\n component userDetailsContainer {}\n compo...,\ta field whose id is 'zw_' with state of type...,* is a child of the form whose id is 'h'.\n* i...,"{""$type"":""Form"",""name"":""h"",""component"":{""$type...",```FormLang\n\n component userDetailsContaine...


##### Example User Prompt vs. original Code

In [ ]:
print(examples['userPrompt'][0])

```FormLang

  component userDetailsContainer {}
  component formContainer {}
  component someOtherContainer {}
  component OtherContainer2 {}


  component myTextBox {
    props {
      textColor
      textSize
      textWeight
      borderColor
    }
  }
  component myCheckbox {
    props {
      size
    }
  }
  component otherTextBox {}
  component counter {
    props {
      style
    }
  }

form i_ {
	comp userDetailsContainer 
	
	form _ {
		comp userDetailsContainer 
		
		field M {
			state string
			comp counter style="(() => 91.82371540078952)()" as expression
			
		}
		field d {
			comp counter style="Math.sqrt(16)" as expression
			
		}
		field Bw {
			state boolean default "true" as expression
			comp otherTextBox 
			
		}
		field j {
			state boolean default "true" as expression
			comp otherTextBox 
			
		}
	}

	form g {
		comp OtherContainer2 
		

	}

	field dw {
		comp myCheckbox size="#342508"
		
	}
}

```
The description of the form element you need to add:
	a field w

In [ ]:
print(examples['originalFormCode'][0])


  component userDetailsContainer {}
  component formContainer {}
  component someOtherContainer {}
  component OtherContainer2 {}


  component myTextBox {
    props {
      textColor
      textSize
      textWeight
      borderColor
    }
  }
  component myCheckbox {
    props {
      size
    }
  }
  component otherTextBox {}
  component counter {
    props {
      style
    }
  }

form i_ {
	comp userDetailsContainer 
	
	form _ {
		comp userDetailsContainer 
		
		field M {
			state string
			comp counter style="(() => 91.82371540078952)()" as expression
			
		}
		field d {
			comp counter style="Math.sqrt(16)" as expression
			
		}
		field Bw {
			state boolean default "true" as expression
			comp otherTextBox 
			
		}
		field j {
			state boolean default "true" as expression
			comp otherTextBox 
			
		}
	}

	form g {
		comp OtherContainer2 
		

	}

	field dw {
		comp myCheckbox size="#342508"
		
	}
	field Ow {
		state string
		comp counter style="#4a6461"
		
	}
}



### Creating a Dataset on HuggingFace using `Datasets`

### Zero-shot evaluation using Llama 3.2-1B

The LLM might ignore the instruction to output plain code without the "```FormLang" delimiter, in that case we should try to extract the contents.

In [ ]:
def extract_code_block(text, lang):
  """
  Extracts a code block from a string.

  Args:
    text: The string containing the code block.
    lang: The name of the language the code block is written in.

  Returns:
    The code block, or None if no code block is found.
  """
  pattern = f"```{lang}\n(.*?)```"
  match = re.search(pattern, text, re.DOTALL)
  if match:
    return match.group(1).strip()
  return None

def extract_llm_formlang_output(text):
    # Strip leading and trailing whitespace and check for delimiters
    text = text.strip()
    return extract_code_block(text, "FormLang") or text

In [ ]:
llama3_pipeline = pipeline("text-generation", model="meta-llama/Llama-3.2-1B-Instruct", torch_dtype=torch.bfloat16, device_map="cpu")

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
%%time
pred_example_0 = llama3_pipeline([
    {"role": "system", "content": "You are a code generation AI assistant.\n" + get_system_prompt()},
    {"role": "user", "content": examples['userPrompt'][0]},
], max_new_tokens=10**4)
pred_example_0

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


CPU times: user 5min 50s, sys: 278 ms, total: 5min 51s
Wall time: 43.9 s


[{'generated_text': [{'role': 'system',
    'content': "You are a code generation AI assistant.\nYour job is to generate valid FormLang code according to the instructions given below:\nInspect the following FormLang Form definition, the start of the code will be denoted with ```FormLang and its end with ``` .\nAfter inspection, complete the Form's code according to the given a description of a new form element and a description of its location in the form.\nYou may assume that the new form element to be added is always either a 'form' or a 'field'.\nYour output must be valid and compiler-friendly FormLang code only.\nIf you are unsure of the FormLang syntax, try to infer it from the form code which is given below as an input.Your answer will be evaluated using an AST comparison of your code to the expected code.\nAssume that the input code is valid and requires no modification other than the NEW code you must generate.\nYou must output plain FormLang code without any additional text or

In [ ]:
pred_example_0_generated = pred_example_0[0]['generated_text'][2]['content']
print(extract_llm_formlang_output(pred_example_0_generated))

component userDetailsContainer {}
  component formContainer {}
  component someOtherContainer {}
  component OtherContainer2 {}


  component myTextBox {
    props {
      textColor
      textSize
      textWeight
      borderColor
    }
  }
  component myCheckbox {
    props {
      size
    }
  }
  component otherTextBox {}
  component counter {
    props {
      style
    }
  }

form i_ {
	comp userDetailsContainer 
	
  field M {
    state string
    comp counter style="(() => 91.82371540078952)()" as expression
  }
  field d {
    comp counter style="Math.sqrt(16)" as expression
  }
  field Bw {
    state boolean default "true" as expression
    comp otherTextBox 
  }
  field j {
    state boolean default "true" as expression
    comp otherTextBox 
  }
  
  field Ow {
    comp myCheckbox
    props {
      size
    }
  }
}

form g {
  comp OtherContainer2 
  

  field dw {
    comp myCheckbox size="#4a6461"
  }
}


In [ ]:
get_ast(extract_llm_formlang_output(pred_example_0_generated), True)

{'ast': '{"$type":"Form","name":"i_","component":{"$type":"FieldComponentDef","componentId":{"$ref":"#/components@0"},"componentPropsKeys":[],"componentPropsValues":[]},"children":[{"$type":"Field","name":"M","state":{"$type":"FieldStateDef","type":"string","isArray":false},"component":{"$type":"FieldComponentDef","componentId":{"$ref":"#/components@7"},"componentPropsKeys":[{"$type":"ComponentPropKey","key":"style"}],"componentPropsValues":[{"$type":"ValueExpression","value":"(() => 91.82371540078952)()","isExpression":true}]}},{"$type":"Field","name":"d","component":{"$type":"FieldComponentDef","componentId":{"$ref":"#/components@7"},"componentPropsKeys":[{"$type":"ComponentPropKey","key":"style"}],"componentPropsValues":[{"$type":"ValueExpression","value":"Math.sqrt(16)","isExpression":true}]}},{"$type":"Field","name":"Bw","state":{"$type":"FieldStateDef","type":"boolean","defaultValue":{"$type":"ValueExpression","value":"true","isExpression":true},"isArray":false},"component":{"$ty